In [43]:
import os
import sys
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import TextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
import pandas as pd
import networkx as nx
import math

os.chdir('/Users/mac/AIworkspace/LLMWORKSPACE/RAG_Rec')
# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Raptor with rec
1. target user의 최신 상호작용 목록을 input query로 넣음 -> 몇개 넣을지는 바꿔가며
2. vectordb는 모든 사용자를 연결시켜둔 db에서 계층적 클러스터링을 시도함 
3. 갑작스럽게 평점 및 영화의 장르가 변화하는 지점에서 끊어 각 상호작용의 패턴을 header로 만듦 
4. 가장 하단 계층의 클러스터와 비교하여 유사도가 임계값을 넘는 지점 or 유사도의 변화율이 급격히 변화하는 시점에서 검색 중단 
5. 해당되는 검색 시점의 클러스터에 있는 사용자들만을 top-100 뽑아냄 

In [8]:
# 영화기록 데이터 
import pandas as pd
file_path = "data/movies.dat"
df2 = pd.read_csv(file_path, delimiter="::", engine="python", header=None,encoding="latin1")
df2.columns = ["MovieID", "Title", "Genres"]

file_path = "data/ratings.dat"
df = pd.read_csv(file_path, delimiter="::", engine="python", header=None,encoding="latin1")
df.columns = ["UserId", "MovieID", "Ratings","timestamp"]
new_df=df.merge(df2, on='MovieID')
df_sorted = new_df.sort_values(by=['UserId', 'timestamp']).reset_index(drop=True)


In [9]:
df_sorted

,UserId,MovieID,Ratings,timestamp,Title,Genres
0,1,3186,4,978300019,"Girl, Interrupted (1999)",Drama
1,1,1270,5,978300055,Back to the Future (1985),Comedy|Sci-Fi
2,1,1721,4,978300055,Titanic (1997),Drama|Romance
3,1,1022,5,978300055,Cinderella (1950),Animation|Children's|Musical
4,1,2340,3,978300103,Meet Joe Black (1998),Romance
...,...,...,...,...,...,...
1000204,6040,2917,4,997454429,Body Heat (1981),Crime|Thriller
1000205,6040,1921,4,997454464,Pi (1998),Sci-Fi|Thriller
1000206,6040,1784,3,997454464,As Good As It Gets (1997),Comedy|Drama
1000207,6040,161,3,997454486,Crimson Tide (1995),Drama|Thriller|War


In [52]:
# --- 2. 사용자별 interaction 문자열 생성 ---
df_sorted['interaction'] = df_sorted.apply(
    lambda row: f"{row['Genres']} (Rating: {row['Ratings']})", axis=1
)

# 사용자별 interaction 연결
user_interactions = df_sorted.groupby('UserId')['interaction'].apply(' → '.join).reset_index()

# 컬럼명 변경
user_interactions.columns = ['UserId', 'interaction_text']


### 1. meta chunking

In [53]:
import pandas as pd

# --- 1. 청킹을 위한 함수 정의 ---
def chunk_user_interactions(user_df):
    chunks = []
    current_chunk = []
    prev_genres = set()
    prev_rating = None
    
    for idx, row in user_df.iterrows():
        current_genres = set(row['interaction'].split(" (Rating: ")[0].split(", "))  # 장르 추출
        current_rating = int(row['interaction'].split(" (Rating: ")[1].strip(")"))  # 평점 추출
        
        # 첫 번째 행은 무조건 새로운 청크로 시작
        if not current_chunk:
            current_chunk.append(row['interaction'])
            prev_genres = current_genres
            prev_rating = current_rating
            continue
        
        # 청킹 조건 확인
        genre_change = not current_genres.intersection(prev_genres)  # 이전 장르와 겹치는 부분이 없는 경우
        rating_change = abs(current_rating - prev_rating) >= 3  # 평점이 3 이상 차이나는 경우
        
        if genre_change and rating_change:
            chunks.append(" → ".join(current_chunk))  # 이전 청크 저장
            current_chunk = []  # 새로운 청크 시작
            
        # 현재 행을 청크에 추가
        current_chunk.append(row['interaction'])
        prev_genres = current_genres
        prev_rating = current_rating

    # 마지막 청크 추가
    if current_chunk:
        chunks.append(" → ".join(current_chunk))
    
    return chunks

# --- 2. 사용자별 청킹 적용 ---
chunked_interactions = user_interactions.groupby("UserId")["interaction_text"].apply(
    lambda x: chunk_user_interactions(pd.DataFrame({"interaction": x.iloc[0].split(" → ")}))
).reset_index()

# 청크를 DataFrame으로 변환
chunked_interactions = chunked_interactions.explode("interaction_text").reset_index(drop=True)

### 2. header 구성- LLM 활용

In [54]:
ci=chunked_interactions.iloc[:10]

- version1

In [68]:
import openai
import pandas as pd
import os

# --- 1. Set OpenAI API Key ---
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- 2. Strict Few-shot Prompt Template ---
STRICT_PROMPT_TEMPLATE = """
### Example 1:
**Input:**
- Genre Flow: Action (Rating: 5) → Thriller (Rating: 4) → Drama (Rating: 2)
- Characteristics: The user prefers action and thriller genres and tends to give lower ratings to drama films.

**Output:** 
"Prefers action and thriller movies, often giving high ratings. Shows less interest in drama."

### Example 2:
**Input:**
- Genre Flow: Romance (Rating: 4) → Comedy (Rating: 5) → Fantasy (Rating: 3)
- Characteristics: The user enjoys romance and comedy films with lighthearted themes but rates fantasy lower.

**Output:** 
"Enjoys romance and comedy, favoring lighthearted themes. Less interest in fantasy."

### Example 3:
**Input:**
- Genre Flow: Sci-Fi (Rating: 5) → Horror (Rating: 4) → Mystery (Rating: 2)
- Characteristics: Strong preference for sci-fi and horror, with lower ratings for mystery.

**Output:** 
"Strong preference for sci-fi and horror. Mystery receives lower ratings."

### Example 4:
**Input:**
- Genre Flow: Drama (Rating: 3) → War (Rating: 4) → Documentary (Rating: 2)
- Characteristics: Prefers war and drama movies while rating documentaries lower.

**Output:** 
"Prefers war and drama. Less interested in documentaries."
"""

# --- 3. Function to Generate Concise Summaries ---
def generate_summary(chunk_text):
    prompt = f"""
{STRICT_PROMPT_TEMPLATE}

### Input:
- Genre Flow: {chunk_text}
- Characteristics: Summarize the user's movie preferences in **one or two sentences** with a focus on key trends.

### Output:
"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # Use "gpt-4-turbo" if needed
            messages=[
                {"role": "system", "content": "Summarize movie viewing patterns in a **concise and structured** format."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,  # Ensures concise responses
            temperature=0.5  # Reduces randomness
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ OpenAI API Error: {e}")
        return None

# --- 4. Apply Summarization to Chunks ---
def generate_chunk_summaries(chunked_df):
    chunked_df["summary"] = chunked_df["interaction_text"].apply(generate_summary)
    return chunked_df

# --- 5. Run the Process ---
ci2 = generate_chunk_summaries(ci)

/var/folders/gl/6qlsk6x94klgs54qzsg9q1jm0000gn/T/ipykernel_4241/3843887144.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunked_df["summary"] = chunked_df["interaction_text"].apply(generate_summary)


- version2

In [80]:
import openai
import pandas as pd
import os

# --- 1. Set OpenAI API Key ---
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- 2. Optimized Few-shot Prompt Template ---
COMPACT_PROMPT_TEMPLATE = """
### Example 1:
**Input:**
Action (Rating: 5) → Thriller (Rating: 4) → Drama (Rating: 2)

**Output:** 
"Prefers action and thriller. Less interest in drama."

### Example 2:
**Input:**
Romance (Rating: 4) → Comedy (Rating: 5) → Fantasy (Rating: 3)

**Output:** 
"Enjoys romance and comedy. Less interest in fantasy."
"""

# --- 3. Function to Generate Concise Summaries ---
def truncate_genre_flow(chunk_text, max_pairs=5):
    """Truncates genre flow to avoid excessive tokens."""
    genre_pairs = chunk_text.split(" → ")
    if len(genre_pairs) > max_pairs:
        genre_pairs = genre_pairs[:max_pairs]  # Keep only first N pairs
    return " → ".join(genre_pairs)

def generate_summary(chunk_text):
    truncated_text = truncate_genre_flow(chunk_text)

    prompt = f"""
{COMPACT_PROMPT_TEMPLATE}

### Input:
{truncated_text}

**Output:**
"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # Use "gpt-4-turbo" if needed
            messages=[
                {"role": "system", "content": "Summarize movie viewing patterns concisely in a structured format."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=40,  # Ensures short summaries
            temperature=0.5
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ OpenAI API Error: {e}")
        return None

# --- 4. Apply Summarization to Chunks ---
def generate_chunk_summaries(chunked_df):
    chunked_df["summary"] = chunked_df["interaction_text"].apply(generate_summary)
    return chunked_df

c3= chunked_interactions.iloc[:100]
# --- 5. Run the Process ---
c4 = generate_chunk_summaries(c3)

/var/folders/gl/6qlsk6x94klgs54qzsg9q1jm0000gn/T/ipykernel_4241/367101244.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunked_df["summary"] = chunked_df["interaction_text"].apply(generate_summary)


In [81]:
c4

,UserId,interaction_text,summary
0,1,Drama (Rating: 4) → Comedy|Sci-Fi (Rating: 5) ...,"""Enjoys drama, comedy, romance, animation, chi..."
1,2,Action|Adventure (Rating: 4) → Action|Adventur...,"""Enjoys action, adventure, romance, sci-fi, an..."
2,2,Drama|War (Rating: 2),"""Has low interest in both drama and war genres."""
3,2,Drama (Rating: 5) → Comedy|Drama|War (Rating: ...,"""Prefers drama, action, and war genres. Enjoys..."
4,2,Comedy|Thriller (Rating: 1),"""Low interest in both comedy and thriller."""
...,...,...,...
95,15,Comedy (Rating: 1) → Action|Thriller (Rating: ...,"""Enjoys action, thriller, and drama genres. Le..."
96,15,Drama (Rating: 1),"""Less interest in drama."""
97,15,Thriller (Rating: 4) → Thriller (Rating: 4) → ...,"""Enjoys thrillers the most, followed by drama ..."
98,15,Drama (Rating: 4) → Comedy|Drama (Rating: 2) →...,"""Enjoys drama, comedy, and a mix of genres. Mo..."


In [82]:
chunked_interactions

,UserId,interaction_text
0,1,Drama (Rating: 4) → Comedy|Sci-Fi (Rating: 5) ...
1,2,Action|Adventure (Rating: 4) → Action|Adventur...
2,2,Drama|War (Rating: 2)
3,2,Drama (Rating: 5) → Comedy|Drama|War (Rating: ...
4,2,Comedy|Thriller (Rating: 1)
...,...,...
56781,6040,Comedy|Drama (Rating: 5) → Drama (Rating: 3) →...
56782,6040,Drama|Romance (Rating: 2) → Drama (Rating: 4) ...
56783,6040,Comedy (Rating: 4) → Children's|Drama|Fantasy|...
56784,6040,Adventure|Animation|Film-Noir (Rating: 1) → Dr...


In [85]:
data.iloc[0].movie_explain

'[\'3186 (Drama) ratings: 4\', \'1270 (Comedy|Sci-Fi) ratings: 5\', \'1721 (Drama|Romance) ratings: 4\', "1022 (Animation|Children\'s|Musical) ratings: 5", \'2340 (Romance) ratings: 3\', \'1836 (Drama) ratings: 5\', \'3408 (Drama) ratings: 4\', \'2804 (Comedy|Drama) ratings: 5\', \'1207 (Drama) ratings: 4\', \'1193 (Drama) ratings: 5\', \'720 (Animation) ratings: 3\', \'260 (Action|Adventure|Fantasy|Sci-Fi) ratings: 4\', "919 (Adventure|Children\'s|Drama|Musical) ratings: 4", \'608 (Crime|Drama|Thriller) ratings: 4\', \'2692 (Action|Crime|Romance) ratings: 4\', \'1961 (Drama) ratings: 5\', \'2028 (Action|Drama|War) ratings: 5\', \'3105 (Drama) ratings: 5\', \'938 (Musical) ratings: 4\', \'1035 (Musical) ratings: 5\', \'1962 (Drama) ratings: 4\', "2018 (Animation|Children\'s) ratings: 4", \'150 (Drama) ratings: 5\', "1028 (Children\'s|Comedy|Musical) ratings: 5", "1097 (Children\'s|Drama|Fantasy|Sci-Fi) ratings: 4", \'914 (Musical|Romance) ratings: 3\', \'1287 (Action|Adventure|Drama) r